In [1]:
# includes
from PIL import Image
from skimage import io, transform

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import torch.optim as optim

In [2]:
# constansts

project_path = "/home/krys/POLIBUDA/Patterns_database/"


dots_path = project_path + "dots/dot"
paisley_path = project_path + "paisley/paisley"
qua_path = project_path + "quaterfoil/qua"
square_path = project_path + "squares/sq"
strips_path = project_path + "strips/pas"
traingles_path = project_path + "triangles/tr"

dots_target = torch.tensor([[float(1), float(0), float(0), float(0), float(0), float(0)]])
paisley_target = torch.tensor([[float(0), float(1), float(0), float(0), float(0), float(0)]])
qua_target = torch.tensor([[float(0), float(0), float(1), float(0), float(0), float(0)]])
square_target = torch.tensor([[float(0), float(0), float(0), float(1), float(0), float(0)]])
strips_target = torch.tensor([[float(0), float(0), float(0), float(0), float(1), float(0)]])
traingles_target = torch.tensor([[float(0), float(0), float(0), float(0), float(0), float(1)]])

paths = [
    dots_path,
    paisley_path,
    qua_path,
    square_path,
    strips_path,
    traingles_path
]

targets = [
    dots_target,
    paisley_target,
    qua_target,
    square_target,
    strips_target,
    traingles_target
]


def getImageTensor(path, idx):
    
    # loading image from local memory
    
    i_path = path + str(idx) + ".jpg"
    image = io.imread(i_path)
    
    return image
    

# transforms image to grayscale tensor
def myTransform(image):

    transformedImage =  tv.transforms.Compose([
                        tv.transforms.ToPILImage(),
                        tv.transforms.Grayscale(num_output_channels=1),
                        tv.transforms.ToTensor()
                      ])(image)
    
    transformedImage = transformedImage[0]

    return transformedImage

In [3]:
# includes
from PIL import Image
from skimage import io, transform

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import torch.optim as optim

# nn model using pytorch schematic
class PatternRecognitionNet(nn.Module):

    def __init__(self):
        super(PatternRecognitionNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 73 * 73, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 6)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
# creting net, lossfunction, optimizer    
pattNet = PatternRecognitionNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(pattNet.parameters(), lr=0.1) # check learning rate

In [4]:
# training function
def train(image, target):
    
    optimizer.zero_grad()   # zero the gradient buffers
    input = image.view(-1,1,300,300)
    output = pattNet(input)
    print("Target: ",target)
    print("Output: ",output)
    loss = criterion(output, target)
    print("Loss calculation: " , loss)
    loss.backward()
    optimizer.step()    # Does the update

In [5]:
idx = 1
error_count = 0
while(1):
    try:
        
        for k in range(len(paths)):    
            print("============================================================\n")
            print("Training : " + str(idx) + " : " + str(k+1) + "/6")
            print("Path: " + paths[k])
            print("\n")
            image = getImageTensor(paths[k], idx)
            t_image = myTransform(image)
            train(t_image, targets[k])
            print("\n")
        
    except Exception as e:
        print("*******************************************************\n")
        print(e)
        print("\n")
        print("*******************************************************")
        
        error_count = error_count + 1
        
        if error_count > 2:    
            break 
        
    finally:
        idx = idx + 1


Training : 1 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[-0.1395, -0.0191,  0.0842,  0.0197, -0.0065,  0.0189]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.2178, grad_fn=<MseLossBackward>)



Training : 1 : 2/6
Path: /home/krys/POLIBUDA/Patterns_database/paisley/paisley


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[ 0.8587,  0.0240,  0.3173, -0.6222, -0.5033,  0.0242]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.4053, grad_fn=<MseLossBackward>)



Training : 1 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[-0.2158,  0.7774,  0.3222,  0.2677,  0.3101,  0.0049]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.2130, grad_fn=<MseLossBackward>)



Training : 1 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Ou




Training : 5 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[0.1774, 0.1828, 0.2181, 0.2103, 0.2643, 0.2159]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1530, grad_fn=<MseLossBackward>)



Training : 5 : 2/6
Path: /home/krys/POLIBUDA/Patterns_database/paisley/paisley


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[0.1795, 0.0916, 0.0764, 0.1447, 0.1155, 0.1502]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1533, grad_fn=<MseLossBackward>)



Training : 5 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[0.1207, 0.1211, 0.0873, 0.1129, 0.0956, 0.1055]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1492, grad_fn=<MseLossBackward>)



Training : 5 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[




Training : 9 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[0.1315, 0.1644, 0.1480, 0.1551, 0.1485, 0.1348]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1391, grad_fn=<MseLossBackward>)



Training : 9 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1423, 0.1741, 0.2014, 0.1554, 0.1462, 0.1430]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1411, grad_fn=<MseLossBackward>)



Training : 9 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1245, 0.1609, 0.1991, 0.1986, 0.1364, 0.1139]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1465, grad_fn=<MseLossBackward>)



Training : 9 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0




Training : 13 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1401, 0.1589, 0.1718, 0.1371, 0.1485, 0.1492]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1439, grad_fn=<MseLossBackward>)



Training : 13 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1237, 0.1408, 0.1610, 0.1727, 0.1320, 0.1333]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1437, grad_fn=<MseLossBackward>)



Training : 13 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0.1129, 0.1261, 0.1543, 0.1628, 0.1716, 0.1179]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1478, grad_fn=<MseLossBackward>)



Training : 14 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[0.1




Training : 17 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.1825, 0.1661, 0.2092, 0.1509, 0.1425, 0.1423]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1444, grad_fn=<MseLossBackward>)



Training : 17 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.1313, 0.1441, 0.1469, 0.1586, 0.1392, 0.1423]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1410, grad_fn=<MseLossBackward>)



Training : 17 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0., 0., 1.]])
Output:  tensor([[0.1174, 0.1263, 0.1300, 0.1425, 0.1740, 0.1370]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.1403, grad_fn=<MseLossBackward>)



Training : 18 : 1/6
Path: /home/krys/POLIBUDA/Patterns_database/dots/dot


Target:  tensor([[1., 0., 0., 0., 0., 0.]])
Output:  tensor([[0.1

...




Training : 1147 : 2/6
Path: /home/krys/POLIBUDA/Patterns_database/paisley/paisley


Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[-0.0060,  0.9579,  0.0506, -0.0485,  0.0319, -0.0042]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0013, grad_fn=<MseLossBackward>)



Training : 1147 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[-2.7539e-02,  1.8023e-01,  7.5214e-01,  1.8753e-02,  4.7824e-02,
         -1.8503e-04]], grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0162, grad_fn=<MseLossBackward>)



Training : 1147 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[0.0288, 0.0298, 0.0158, 0.8854, 0.0220, 0.0046]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0026, grad_fn=<MseLossBackward>)



Training : 1147 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  t

Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[-0.0518,  1.1133, -0.0683, -0.1116,  0.0345, -0.0639]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0063, grad_fn=<MseLossBackward>)



Training : 1151 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[ 0.0242,  0.0273,  0.9684, -0.0117,  0.0329,  0.0025]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0006, grad_fn=<MseLossBackward>)



Training : 1151 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[ 0.0313, -0.0084,  0.0337,  0.8925,  0.0054,  0.0186]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0024, grad_fn=<MseLossBackward>)



Training : 1151 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[-0.0258, -0.0191,  0.0354,  0.0578,  0.9137,  0.0632]],

Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[ 2.6740e-02,  9.6280e-01,  2.0912e-02, -1.5279e-02,  4.5314e-04,
          3.3386e-02]], grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0006, grad_fn=<MseLossBackward>)



Training : 1155 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[-1.2509e-02, -1.4275e-02,  1.0929e+00,  1.2555e-03, -5.2141e-02,
          5.1916e-04]], grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0020, grad_fn=<MseLossBackward>)



Training : 1155 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[-0.0299,  0.0806,  0.1010,  0.9447, -0.0877,  0.0098]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0047, grad_fn=<MseLossBackward>)



Training : 1155 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[-0.




Training : 1159 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[-0.0108,  0.0367,  0.9424, -0.0145,  0.0548, -0.0134]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0014, grad_fn=<MseLossBackward>)



Training : 1159 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[ 0.0099, -0.0061, -0.0215,  0.9481,  0.0355,  0.0111]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0008, grad_fn=<MseLossBackward>)



Training : 1159 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[-0.0120,  0.1058,  0.0309,  0.0745,  0.7824,  0.0094]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0109, grad_fn=<MseLossBackward>)



Training : 1159 : 6/6
Path: /home/krys/POLIBUDA/Patterns_database/triangles/tr


Target:  tensor([[0., 0., 0., 0.,

Target:  tensor([[0., 1., 0., 0., 0., 0.]])
Output:  tensor([[-0.0172,  1.0850, -0.0275, -0.0211,  0.0115, -0.0176]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0015, grad_fn=<MseLossBackward>)



Training : 1163 : 3/6
Path: /home/krys/POLIBUDA/Patterns_database/quaterfoil/qua


Target:  tensor([[0., 0., 1., 0., 0., 0.]])
Output:  tensor([[0.0535, 0.0595, 0.7621, 0.0527, 0.0134, 0.0145]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0110, grad_fn=<MseLossBackward>)



Training : 1163 : 4/6
Path: /home/krys/POLIBUDA/Patterns_database/squares/sq


Target:  tensor([[0., 0., 0., 1., 0., 0.]])
Output:  tensor([[-0.0435,  0.0335,  0.0156,  0.9149,  0.0622, -0.0017]],
       grad_fn=<AddmmBackward>)
Loss calculation:  tensor(0.0024, grad_fn=<MseLossBackward>)



Training : 1163 : 5/6
Path: /home/krys/POLIBUDA/Patterns_database/strips/pas


Target:  tensor([[0., 0., 0., 0., 1., 0.]])
Output:  tensor([[0.0090, 0.0481, 0.1024, 0.0476, 0.8383, 0.0423]],
       grad

In [9]:
PATH = project_path + "/model.pt"
torch.save(pattNet.state_dict(), PATH)